In [1]:
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchtext.data.utils import get_tokenizer
from torchtext.datasets import DATASETS
from torchtext.vocab import build_vocab_from_iterator
import torch.nn as nn
from tqdm import tqdm
import pickle
import random
import numpy as np
from collections import Counter, defaultdict
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
import gensim.downloader
from torch import FloatTensor as FT
import fasttext

# Get the interactive Tools for Matplotlib
%matplotlib notebook
%matplotlib inline

plt.style.use('ggplot')

### Using fastText
 - See here: https://fasttext.cc/docs/en/unsupervised-tutorial.html
 - Available in torchtext: https://torchtext.readthedocs.io/en/latest/vocab.html#fasttext

# Visualize some pretrained models.
- See: https://radimrehurek.com/gensim/models/word2vec.html

In [2]:
# Pretrained models!
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [7]:
model_glove = gensim.downloader.load('glove-wiki-gigaword-300')
model_fast_text = gensim.downloader.load('fasttext-wiki-news-subwords-300')
model_word2vec = gensim.downloader.load('word2vec-google-news-300')

In [8]:
# This takes the vector for a word and gets the most similar words to this vector.
model_glove.most_similar('banana')

[('bananas', 0.6691170930862427),
 ('mango', 0.580410361289978),
 ('pineapple', 0.5492372512817383),
 ('coconut', 0.5462778806686401),
 ('papaya', 0.541056752204895),
 ('fruit', 0.5218108296394348),
 ('growers', 0.4877638816833496),
 ('nut', 0.48399588465690613),
 ('peanut', 0.4806201756000519),
 ('potato', 0.48061180114746094)]

In [9]:
model_fast_text.most_similar('banana')

[('bananas', 0.8146674633026123),
 ('pineapple', 0.730781614780426),
 ('bannana', 0.7231925129890442),
 ('bananaquit', 0.6934816837310791),
 ('mango', 0.6926952004432678),
 ('coconut', 0.6897603273391724),
 ('banana-leaf', 0.679623544216156),
 ('bananna', 0.672265887260437),
 ('banana-like', 0.6696617603302002),
 ('banana-shaped', 0.6661475896835327)]

In [10]:
model_word2vec.most_similar('banana')

[('bananas', 0.752377450466156),
 ('pineapple', 0.6587537527084351),
 ('mango', 0.6365211009979248),
 ('pineapples', 0.6313878297805786),
 ('papaya', 0.6063666343688965),
 ('coconut', 0.6035483479499817),
 ('potato', 0.5747568607330322),
 ('melon', 0.5625776052474976),
 ('cashew', 0.562165379524231),
 ('tomato', 0.5590205788612366)]

In [11]:
model_glove.most_similar('obama')

[('barack', 0.9254721999168396),
 ('mccain', 0.7590768337249756),
 ('bush', 0.7570987343788147),
 ('clinton', 0.7085603475570679),
 ('hillary', 0.6497916579246521),
 ('kerry', 0.6144052743911743),
 ('rodham', 0.6138635277748108),
 ('biden', 0.5940852165222168),
 ('gore', 0.5885976552963257),
 ('democrats', 0.5608304142951965)]

In [12]:
model_fast_text.most_similar('obama')

[('obamas', 0.8718591928482056),
 ('obama.', 0.8496118783950806),
 ('obam', 0.8469557166099548),
 ('0bama', 0.8263689279556274),
 ('obamma', 0.8021047711372375),
 ('nobama', 0.8001623749732971),
 ('obamanation', 0.7925804257392883),
 ('Nobama', 0.7780248522758484),
 ('Obama.', 0.7572956085205078),
 ('Õbama', 0.753665566444397)]

In [14]:
# x2 - x1 = y2 - y1 ... So y2 = x2 - x1 + y1 ... Consider all neighbors of x2 - x1 + y1, get y2. 
def analogy(model, x1, x2, y1):
    result = model.most_similar(positive=[y1, x2], negative=[x1])
    return result[0][0]

In [15]:
analogy(model_glove, 'japan', 'japanese', 'australia')

'australian'

In [16]:
analogy(model_fast_text, 'japan', 'japanese', 'australia')

'australian'

In [17]:
analogy(model_word2vec, 'japan', 'japanese', 'australia')

'canada'

In [18]:
analogy(model_glove, 'obama', 'clinton', 'reagan')

'ronald'

In [20]:
analogy(model_fast_text, 'obama', 'clinton', 'reagan')

'margaret'

In [22]:
analogy(model_glove, 'tall', 'tallest', 'long')

'longest'

In [23]:
analogy(model_fast_text, 'tall', 'tallest', 'long')

'longest'

In [24]:
analogy(model_word2vec, 'tall', 'tallest', 'long')

'longest'

In [ ]:
# Which word from the given list doesn’t go with the others?
print(model_word2vec.doesnt_match("breakfast cereal dinner lunch".split()))

In [ ]:
def display_pca_scatterplot(model, words=None, sample=0):
    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]
        
    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

### Let's project to two dimensions and see if thee are any patterns.
We see some clusters:
- Countries are together.
- Drinks are together.
- Foods.
- Etc.

In [ ]:
display_pca_scatterplot(
    model_fast_text, 
    [
        'coffee', 'tea', 'beer', 'wine', 'brandy', 'rum', 'champagne', 'water',
        'spaghetti', 'borscht', 'hamburger', 'pizza', 'falafel', 'sushi', 'meatballs',
        'dog', 'horse', 'cat', 'monkey', 'parrot', 'koala', 'lizard',
        'frog', 'toad', 'monkey', 'ape', 'kangaroo', 'wombat', 'wolf',
        'france', 'germany', 'hungary', 'france', 'australia', 'fiji', 'china',
        'homework', 'assignment', 'problem', 'exam', 'test', 'class',
        'school', 'college', 'university', 'institute'
    ]
)

### Information
- torchtext repo: https://github.com/pytorch/text/tree/main/torchtext
- torchtext documentation: https://pytorch.org/text/stable/index.html

In [12]:
# Where do I want to run my job. You can do "cuda" on linux machines.
DEVICE = 'cpu'
# The batch size in Adam or SGD.
BATCH_SIZE = 128
# Number of epochs.
NUM_EPOCHS = 10

The text8 Wikipedia corpus. 100M characters.

In [13]:
!du -h text8

 95M	text8


In [14]:
f = open('text8', 'r')
text = f.read()
# One big string of size 100M.
print(len(text))

100000000


In [15]:
punc = '!"#$%&()*+,-./:;<=>?@[\\]^_\'{|}~\t\n'

# Can do regular expressions here too.
for c in punc:
    if c in text:
        text.replace(c, ' ')

In [16]:
# A very crude tokenizer you get for free: lower case and also split on spaces.
TOKENIZER = get_tokenizer("basic_english")

In [17]:
words = TOKENIZER(text)
f = Counter(words)

In [18]:
len(words)

17005207

In [19]:
# Do a very crude filter on the text which remoes all words whuch 
text = [word for word in words if f[word] > 5]

In [20]:
text[0:5]

['anarchism', 'originated', 'as', 'a', 'term']

In [21]:
VOCAB = build_vocab_from_iterator([text])

In [22]:
# word -> int hash map.
stoi = VOCAB.get_stoi()
# int -> word hash map.
itos = VOCAB.get_itos() 

In [23]:
stoi['as']

11

In [24]:
# Total number of words.
len(stoi)

63641

In [25]:
f = Counter(text)
# This is the probability that we pick a word in the corpus.
z = {word: f[word] / len(text) for word in f}

In [26]:
threshold = 1e-5
# Probability that word is kept while subsampling.
# This is explained here and sightly differet from the paper: http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/
p_keep = {word: (np.sqrt(z[word] / 0.001) + 1)*(0.0001 / z[word]) for word in f}

In [27]:
# This is in the integer space.
train_dataset = [word for word in text if random.random() < p_keep[word]]

# Rebuild the vocabulary.
VOCAB = build_vocab_from_iterator([train_dataset])

In [28]:
# word -> int mapping.
stoi = VOCAB.get_stoi()
# int -> word mapping.
itos = VOCAB.get_itos() 

In [29]:
# The vocabulary size after we do all the filters.
len(VOCAB)

63641

In [ ]:
# The probability we draw something for negative sampling.
f = Counter(train_dataset)
p = torch.zeros(len(VOCAB))

# Downsample frequent words and upsample less frequent.
s = sum([np.power(freq, 0.75) for word, freq in f.items()])        

for word in f:
    p[stoi[word]] = np.power(f[word], 0.75) / s

In [60]:
# Map everything to integers.
train_dataset = [stoi[word] for word in text]

In [61]:
# This just gets the (wc, wo) pairs that are positive - they are seen together!
def get_tokenized_dataset(dataset, verbose=False):
    pairs_to_x = defaultdict(int)
    x_list = []
    x_max = 100
    alpha = 3.0 / 4
        
    for idx, i in enumerate(dataset):
        # Pick a context of 10 words to the left/right.
        w = 10
        
        start = max(0,idx-w)
        
        end = min(idx+w,len(dataset)-1)
        
        left_context = dataset[start:idx]
        right_context = dataset[idx+1:end+1]
        
        for jdx, j in enumerate(left_context[::-1]):
            pairs_to_x[(i, j)] += 1.0 / (jdx + 1)
            
        for jdx, j in enumerate(right_context):
            pairs_to_x[(i, j)] += 1.0 / (jdx + 1)
                        
    for token_pair, x in pairs_to_x.items():
        x_list.append(
            list(token_pair) + [(x / x_max) ** alpha if x < x_max else 1.0] + [np.log(x + 1.0)]
        )
    
    # We'll want to batch this.
    return x_list

In [62]:
train_x_list = get_tokenized_dataset(train_dataset, verbose=False)

In [64]:
pickle.dump(train_x_list, open('train_x_list.pkl', 'wb'))

In [31]:
train_x_list = pickle.load(open('train_x_list.pkl', 'rb'))

In [63]:
# This is the (wc, wo, f_X, log_X) data .
train_x_list[:10]

[[5233, 3083, 0.03162277660168379, 0.6931471805599453],
 [5233, 11, 0.4355462302836717, 3.5268039370319726],
 [5233, 5, 0.4860823988912657, 3.6691625374302115],
 [5233, 170, 0.07305473818939927, 1.3996962172069063],
 [5233, 1, 0.9163990258075021, 4.499937528117311],
 [5233, 3133, 0.00824872000639961, 0.15415067982725836],
 [5233, 47, 0.03501746331316858, 0.7634355088765358],
 [5233, 54, 0.02992269147363569, 0.6569852765399865],
 [5233, 139, 0.03076240110322577, 0.674926634446417],
 [5233, 127, 0.03224192659011195, 0.7061574172959602]]

In [66]:
# The number of things of BATCH_SIZE = 256.
len(train_x_list) // 256

177686

### Set up the dataloader.

In [67]:
BATCH_SIZE = 256

train_dl = DataLoader(
    TensorDataset(
        torch.tensor(train_x_list, dtype=torch.float32).to(DEVICE),
    ),
    batch_size=BATCH_SIZE,
    shuffle=True
)

### Words we'll use to asses the quality of the model ...

In [68]:
valid_ids = torch.tensor([
    stoi['money'],
    stoi['lion'],
    stoi['africa'],
    stoi['musician'],
    stoi['dance'],
])

### Get the model.

In [69]:
class GloVe(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(GloVe, self).__init__()
        self.a = nn.Embedding(vocab_size, embed_dim)
        self.c = nn.Embedding(vocab_size, 1)
        self.b = nn.Embedding(vocab_size, embed_dim)
        self.d = nn.Embedding(vocab_size, 1)
        self.x_max = 100
        self.alpha = 3.0 / 4
        
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.w.weight.data.uniform_(-initrange, initrange)
        self.w_tilde.weight.data.uniform_(-initrange, initrange)
        self.b.weight.data.uniform_(-initrange, initrange)
        self.b_tilde.weight.data.uniform_(-initrange, initrange)

    def forward(self, pairs):
        targets = pairs[:, 0]
        contexts = pairs[:, 1]
        
        out = self.a(targets) * self.b(contexts) + self.c(targets)  + self.d(contexts)
        
        out = out.sum(axis=1)
        
        return out
    
def glove_criterion(similarity, log_X, f_X):
    return (f_X * (similarity - log_X) ** 2).mean()

In [70]:
@torch.no_grad()
def validate_embeddings(
    model,
    valid_ids,
    itos
):
    """ Validation logic """

    # Use the sum of the embeddings as the word embedding.
    embedding_weights = model.a.weight + model.b.weight
        
    normalized_embeddings = embedding_weights.cpu() / np.sqrt(
        np.sum(embedding_weights.cpu().numpy()**2, axis=1, keepdims=True)
    )

    # Get the embeddings corresponding to valid_term_ids
    valid_embeddings = normalized_embeddings[valid_ids, :]

    # Compute the similarity between valid_term_ids (S) and all the embeddings (V).
    # We do S x d (d x V) => S x D and sort by negative similarity.
    top_k = 10 # Top k items will be displayed.
    similarity = np.dot(valid_embeddings.cpu().numpy(), normalized_embeddings.cpu().numpy().T)

    # Invert similarity matrix to negative
    # Ignore the first one because that would be the same word as the probe word
    similarity_top_k = np.argsort(-similarity, axis=1)[:, 1: top_k+1]

    # Print the output.
    for i, word_id in enumerate(valid_ids):
        # j >= 1 here since we don't want to include the word itself.
        similar_word_str = ', '.join([itos[j] for j in similarity_top_k[i, :] if j >= 1])
        print(f"{itos[word_id]}: {similar_word_str}")

    print('\n')

### Set up the model

In [71]:
LR = 0.001
NUM_EPOCHS = 10
EMBED_DIM = 100

In [72]:
model = GloVe(len(VOCAB), EMBED_DIM).to(DEVICE)
criterion = glove_criterion

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# The learning rate is lowered every epoch by 1/10.
# Is this a good idea?

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)

In [73]:
model

GloVe(
  (w): Embedding(63641, 100)
  (b): Embedding(63641, 1)
  (w_tilde): Embedding(63641, 100)
  (b_tilde): Embedding(63641, 1)
)

In [74]:
validate_embeddings(model, valid_ids, itos)

money: dfts, roald, bathed, firsts, lining, starling, buyoya, ostend, ionospheric, osprey
lion: mawson, katzenjammer, vital, agra, beekeepers, implementation, exploding, punjabi, kaolinite, antoni
africa: unanimous, superbike, katie, armistead, patriotic, regulations, governing, prenatal, regency, excepting
musician: erd, crossrail, handheld, ruthenia, abstinence, chestnuts, locarno, wyatt, favouring, landsmannschaft
dance: mvs, predict, soundhole, toffler, southwest, creationists, clairvoyance, titisee, momentum, muff




### Train the model

In [75]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_loss, total_batches = 0.0, 0.0
    log_interval = 100

    # Loop through each batch.
    for idx, pairs_X in enumerate(dataloader):
        
        pairs_X = pairs_X[0]
        
        pairs, f_X, log_X = pairs_X[:,:2].int(), pairs_X[:, 2], pairs_X[:, -1]
        
        optimizer.zero_grad()
        
        similarity = model(pairs)     
                
        # Get the loss.
        loss = criterion(similarity, f_X, log_X)
                
        # Do back propagation.
        loss.backward()
                        
        # Clip the gradients.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        
        # Do an optimization step.
        optimizer.step()
        
        total_loss += loss.item()
        total_batches += 1
                
        if idx % log_interval == 0 and idx > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| loss {:8.3f} ".format(
                    epoch,
                    idx,
                    len(dataloader),
                    total_loss / total_batches
                )
            )
            if idx % 10000 == 0 and idx > 0:
                validate_embeddings(model, valid_ids, itos)
            total_loss, total_batches = 0.0, 0.0

### Some results from the run look like below:


In [76]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    
    train(train_dl, model, optimizer, criterion, epoch)
    # We have a learning rate scheduler here.
    # Basically, given the state of the optimizer, this lowers the learning rate in a smart way.
    scheduler.step()

| epoch   1 |   100/177687 batches | loss  779.439 
| epoch   1 |   200/177687 batches | loss  721.320 
| epoch   1 |   300/177687 batches | loss  665.818 
| epoch   1 |   400/177687 batches | loss  631.914 
| epoch   1 |   500/177687 batches | loss  595.464 
| epoch   1 |   600/177687 batches | loss  579.153 
| epoch   1 |   700/177687 batches | loss  539.542 
| epoch   1 |   800/177687 batches | loss  505.596 
| epoch   1 |   900/177687 batches | loss  478.929 
| epoch   1 |  1000/177687 batches | loss  447.031 
| epoch   1 |  1100/177687 batches | loss  433.600 
| epoch   1 |  1200/177687 batches | loss  414.378 
| epoch   1 |  1300/177687 batches | loss  381.672 
| epoch   1 |  1400/177687 batches | loss  367.146 
| epoch   1 |  1500/177687 batches | loss  348.813 
| epoch   1 |  1600/177687 batches | loss  336.486 
| epoch   1 |  1700/177687 batches | loss  320.846 
| epoch   1 |  1800/177687 batches | loss  301.393 
| epoch   1 |  1900/177687 batches | loss  286.610 
| epoch   1 

| epoch   1 | 14900/177687 batches | loss   12.310 
| epoch   1 | 15000/177687 batches | loss   12.150 
| epoch   1 | 15100/177687 batches | loss   11.597 
| epoch   1 | 15200/177687 batches | loss   12.017 
| epoch   1 | 15300/177687 batches | loss   12.378 
| epoch   1 | 15400/177687 batches | loss   10.788 
| epoch   1 | 15500/177687 batches | loss   11.421 
| epoch   1 | 15600/177687 batches | loss   10.789 
| epoch   1 | 15700/177687 batches | loss   10.483 
| epoch   1 | 15800/177687 batches | loss   10.583 
| epoch   1 | 15900/177687 batches | loss    9.779 
| epoch   1 | 16000/177687 batches | loss   10.253 
| epoch   1 | 16100/177687 batches | loss   10.007 
| epoch   1 | 16200/177687 batches | loss   10.220 
| epoch   1 | 16300/177687 batches | loss    9.201 
| epoch   1 | 16400/177687 batches | loss    9.690 
| epoch   1 | 16500/177687 batches | loss    8.881 
| epoch   1 | 16600/177687 batches | loss    9.363 
| epoch   1 | 16700/177687 batches | loss    8.963 
| epoch   1 

| epoch   1 | 29700/177687 batches | loss    1.887 
| epoch   1 | 29800/177687 batches | loss    2.002 
| epoch   1 | 29900/177687 batches | loss    1.811 
| epoch   1 | 30000/177687 batches | loss    1.876 
money: bathed, starling, labienus, quarrelled, helena, ionospheric, sensibilities, format, skilled, scholz
lion: agra, originality, intoxicating, yoghurt, implementation, stacked, kaolinite, vital, eschrichtius, isaac
africa: tzen, patriotic, viktor, excepting, impact, nipples, mondays, minbari, joost, waxing
musician: wyatt, davy, stanfield, orinoco, cypher, kristallnacht, graduated, inf, roly, crossrail
dance: legislatures, titisee, episodes, spreadsheets, muff, clairvoyance, avalon, momentum, vai, enlists


| epoch   1 | 30100/177687 batches | loss    1.681 
| epoch   1 | 30200/177687 batches | loss    1.770 
| epoch   1 | 30300/177687 batches | loss    1.925 
| epoch   1 | 30400/177687 batches | loss    1.875 
| epoch   1 | 30500/177687 batches | loss    2.136 
| epoch   1 | 30

| epoch   1 | 43500/177687 batches | loss    0.828 
| epoch   1 | 43600/177687 batches | loss    0.835 
| epoch   1 | 43700/177687 batches | loss    0.881 
| epoch   1 | 43800/177687 batches | loss    0.854 
| epoch   1 | 43900/177687 batches | loss    0.875 
| epoch   1 | 44000/177687 batches | loss    0.837 
| epoch   1 | 44100/177687 batches | loss    0.876 
| epoch   1 | 44200/177687 batches | loss    0.793 
| epoch   1 | 44300/177687 batches | loss    0.887 
| epoch   1 | 44400/177687 batches | loss    0.933 
| epoch   1 | 44500/177687 batches | loss    0.774 
| epoch   1 | 44600/177687 batches | loss    0.835 
| epoch   1 | 44700/177687 batches | loss    0.805 
| epoch   1 | 44800/177687 batches | loss    0.790 
| epoch   1 | 44900/177687 batches | loss    0.777 
| epoch   1 | 45000/177687 batches | loss    0.802 
| epoch   1 | 45100/177687 batches | loss    0.779 
| epoch   1 | 45200/177687 batches | loss    0.818 
| epoch   1 | 45300/177687 batches | loss    0.814 
| epoch   1 

| epoch   1 | 58300/177687 batches | loss    0.754 
| epoch   1 | 58400/177687 batches | loss    0.664 
| epoch   1 | 58500/177687 batches | loss    0.688 
| epoch   1 | 58600/177687 batches | loss    0.661 
| epoch   1 | 58700/177687 batches | loss    0.691 
| epoch   1 | 58800/177687 batches | loss    0.758 
| epoch   1 | 58900/177687 batches | loss    0.650 
| epoch   1 | 59000/177687 batches | loss    0.678 
| epoch   1 | 59100/177687 batches | loss    0.661 
| epoch   1 | 59200/177687 batches | loss    0.708 
| epoch   1 | 59300/177687 batches | loss    0.688 
| epoch   1 | 59400/177687 batches | loss    0.661 
| epoch   1 | 59500/177687 batches | loss    0.704 
| epoch   1 | 59600/177687 batches | loss    0.687 
| epoch   1 | 59700/177687 batches | loss    0.651 
| epoch   1 | 59800/177687 batches | loss    0.702 
| epoch   1 | 59900/177687 batches | loss    0.668 
| epoch   1 | 60000/177687 batches | loss    0.668 
money: roman, selkirk, hultsfred, synergy, interrelationships, c

...

| epoch  10 | 106100/177687 batches | loss    0.016 
| epoch  10 | 106200/177687 batches | loss    0.016 
| epoch  10 | 106300/177687 batches | loss    0.017 
| epoch  10 | 106400/177687 batches | loss    0.017 
| epoch  10 | 106500/177687 batches | loss    0.018 
| epoch  10 | 106600/177687 batches | loss    0.018 
| epoch  10 | 106700/177687 batches | loss    0.018 
| epoch  10 | 106800/177687 batches | loss    0.018 
| epoch  10 | 106900/177687 batches | loss    0.016 
| epoch  10 | 107000/177687 batches | loss    0.017 
| epoch  10 | 107100/177687 batches | loss    0.016 
| epoch  10 | 107200/177687 batches | loss    0.018 
| epoch  10 | 107300/177687 batches | loss    0.017 
| epoch  10 | 107400/177687 batches | loss    0.016 
| epoch  10 | 107500/177687 batches | loss    0.016 
| epoch  10 | 107600/177687 batches | loss    0.016 
| epoch  10 | 107700/177687 batches | loss    0.016 
| epoch  10 | 107800/177687 batches | loss    0.018 
| epoch  10 | 107900/177687 batches | loss    

| epoch  10 | 120100/177687 batches | loss    0.016 
| epoch  10 | 120200/177687 batches | loss    0.017 
| epoch  10 | 120300/177687 batches | loss    0.018 
| epoch  10 | 120400/177687 batches | loss    0.016 
| epoch  10 | 120500/177687 batches | loss    0.017 
| epoch  10 | 120600/177687 batches | loss    0.017 
| epoch  10 | 120700/177687 batches | loss    0.016 
| epoch  10 | 120800/177687 batches | loss    0.018 
| epoch  10 | 120900/177687 batches | loss    0.018 
| epoch  10 | 121000/177687 batches | loss    0.016 
| epoch  10 | 121100/177687 batches | loss    0.016 
| epoch  10 | 121200/177687 batches | loss    0.016 
| epoch  10 | 121300/177687 batches | loss    0.017 
| epoch  10 | 121400/177687 batches | loss    0.018 
| epoch  10 | 121500/177687 batches | loss    0.018 
| epoch  10 | 121600/177687 batches | loss    0.017 
| epoch  10 | 121700/177687 batches | loss    0.017 
| epoch  10 | 121800/177687 batches | loss    0.016 
| epoch  10 | 121900/177687 batches | loss    

| epoch  10 | 134800/177687 batches | loss    0.017 
| epoch  10 | 134900/177687 batches | loss    0.017 
| epoch  10 | 135000/177687 batches | loss    0.016 
| epoch  10 | 135100/177687 batches | loss    0.016 
| epoch  10 | 135200/177687 batches | loss    0.018 
| epoch  10 | 135300/177687 batches | loss    0.016 
| epoch  10 | 135400/177687 batches | loss    0.016 
| epoch  10 | 135500/177687 batches | loss    0.018 
| epoch  10 | 135600/177687 batches | loss    0.017 
| epoch  10 | 135700/177687 batches | loss    0.017 
| epoch  10 | 135800/177687 batches | loss    0.016 
| epoch  10 | 135900/177687 batches | loss    0.016 
| epoch  10 | 136000/177687 batches | loss    0.027 
| epoch  10 | 136100/177687 batches | loss    0.017 
| epoch  10 | 136200/177687 batches | loss    0.016 
| epoch  10 | 136300/177687 batches | loss    0.017 
| epoch  10 | 136400/177687 batches | loss    0.018 
| epoch  10 | 136500/177687 batches | loss    0.018 
| epoch  10 | 136600/177687 batches | loss    

| epoch  10 | 149500/177687 batches | loss    0.023 
| epoch  10 | 149600/177687 batches | loss    0.018 
| epoch  10 | 149700/177687 batches | loss    0.016 
| epoch  10 | 149800/177687 batches | loss    0.016 
| epoch  10 | 149900/177687 batches | loss    0.016 
| epoch  10 | 150000/177687 batches | loss    0.018 
money: house, special, forces, u, come, came, men, k, political, under
lion: waveguide, lilies, authorised, didgeridoo, durr, combs, judged, banning, indentured, purist
africa: get, upon, government, n, national, men, b, along, together, born
musician: hadith, nzwani, nipples, corps, ritualistic, mcfowl, os, bolted, lipton, academy
dance: inelastic, philosophy, page, programming, khadjimba, novels, controversial, nothing, delray, except


| epoch  10 | 150100/177687 batches | loss    0.018 
| epoch  10 | 150200/177687 batches | loss    0.017 
| epoch  10 | 150300/177687 batches | loss    0.017 
| epoch  10 | 150400/177687 batches | loss    0.016 
| epoch  10 | 150500/177687

| epoch  10 | 163300/177687 batches | loss    0.018 
| epoch  10 | 163400/177687 batches | loss    0.017 
| epoch  10 | 163500/177687 batches | loss    0.016 
| epoch  10 | 163600/177687 batches | loss    0.016 
| epoch  10 | 163700/177687 batches | loss    0.019 
| epoch  10 | 163800/177687 batches | loss    0.017 
| epoch  10 | 163900/177687 batches | loss    0.017 
| epoch  10 | 164000/177687 batches | loss    0.017 
| epoch  10 | 164100/177687 batches | loss    0.016 
| epoch  10 | 164200/177687 batches | loss    0.016 
| epoch  10 | 164300/177687 batches | loss    0.017 
| epoch  10 | 164400/177687 batches | loss    0.016 
| epoch  10 | 164500/177687 batches | loss    0.019 
| epoch  10 | 164600/177687 batches | loss    0.017 
| epoch  10 | 164700/177687 batches | loss    0.015 
| epoch  10 | 164800/177687 batches | loss    0.017 
| epoch  10 | 164900/177687 batches | loss    0.017 
| epoch  10 | 165000/177687 batches | loss    0.017 
| epoch  10 | 165100/177687 batches | loss    